In [113]:
import soundfile as sf

def clip_audio(input_file, output_file, new_start, new_end):
    # Read the input FLAC audio file
    audio, sample_rate = sf.read(input_file)
    strach = 0.28 # in [sec]

    # Convert start and end times from seconds to samples
    # strach the end if possible
    new_end = new_end + strach if ((new_end+strach)*sample_rate <= len(audio))\
                               else -1
    start_sample = int(new_start * sample_rate)
    end_sample = int(new_end * sample_rate)

    # Clip the audio between start and end samples
    if new_end == -1:
      clipped_audio = audio[start_sample:]
    else:
      clipped_audio = audio[start_sample:end_sample]

    # Save the clipped audio to a new FLAC file
    sf.write(output_file, clipped_audio, sample_rate)

def remove_non_alphabetic(input_string):
    # Initialize an empty string to store the result
    result = ""

    # Iterate over each character in the input string
    for char in input_string:
        # Check if the character is an alphabet letter
        if char.isalpha():
            # If it is, append it to the result string
            result += char
    return result


In [1]:
import whisper
import os
from remove_redundant_records import words_to_check 

model = whisper.load_model("base")
for file_name in os.listdir("/content/dev_set_clean"):
  if file_name.endswith(".flac"):
    transcript = model.transcribe(word_timestamps=True, audio=f"/content/dev_set_clean/{file_name}")
    for segment in transcript["segments"]:
        for word in segment["words"]:
            clean_word = remove_non_alphabetic(word["word"])
            new_start = float(word['start'])
            new_end = float(word['end'])
            if clean_word in words_to_check:
              print(f"|word:{clean_word}|start:{new_start}|end:{new_end}|")
              clip_audio(input_file=f"/content/dev_set_clean/{file_name}",
                        output_file=f"/content/dev_set_clean/clipped-{file_name}",
                        new_start=new_start, new_end=new_end)
              break



ModuleNotFoundError: No module named 'whisper'